In [7]:
# class for the dataset:

# class for text-datasets
# Jama, NEJM and lancet
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __len__(self):
        return len(self.total_examples)
        pass

    def __init__(self, string, text_file):
        self.string = string
        self.text_file = text_file

        self.training_examples = []

        total_inputs = []
        total_outputs = []
        with open(text_file) as file:
            for line in file:
                # zero pad to start
                inputs = [None] + [x for x in line]
                targets = inputs[1:] + ["\n"]  # we need a 0 as well!

                total_inputs.extend(inputs)
                total_outputs.extend(targets)

        assert len(total_inputs) == len(total_outputs)
        self.total_examples = list(zip(total_inputs, total_outputs))
        
        #                 for char in line:
        # how many chars before the target? or all chars before the target?
        # so we simply pair up x with [x+1] i think
        # ok, and then we iterate them in the loader?
        #                     inputs = [x for x in ]
        #                     targets = []

        # we should also have some big list of entries 

        pass

    # number of items in the dataset. This involves a rather large computation!
    # we can either do a feed forward cllm in the style of bengio, or we can just 
    # do the RNN approach (as specified by Sean Robertson in his tutorial)

    # the indices should represent examples inside the list
    # one strange thing is the following: we need to precompute al the indices earlier, imo, and then simply return the list
    # at that element! (perhaps with some transforms!)
    def __getitem__(self, index):
        return self.total_examples[index]

        # build a list, return elements from that list? we need to internally keep track of some index
        pass



# ok, so it works now!

In [8]:
abc = TextDataset("rand", "data\\names\\English.txt")


In [9]:
for (a,b) in abc:
    print(a,b)
    
#     realistically, we should also have it as tensors/vectors
# finally, we should also have some way where we run the rnn forward and 

None A
A b
b b
b a
a s
s 


 

None A
A b
b b
b e
e y
y 


 

None A
A b
b b
b o
o t
t t
t 


 

None A
A b
b d
d i
i 


 

None A
A b
b e
e l
l 


 

None A
A b
b r
r a
a h
h a
a m
m 


 

None A
A b
b r
r a
a h
h a
a m
m s
s 


 

None A
A b
b r
r a
a m
m s
s 


 

None A
A c
c k
k a
a r
r y
y 


 

None A
A c
c k
k r
r o
o y
y d
d 


 

None A
A c
c t
t o
o n
n 


 

None A
A d
d a
a i
i r
r 


 

None A
A d
d a
a m
m 


 

None A
A d
d a
a m
m s
s 


 

None A
A d
d a
a m
m s
s o
o n
n 


 

None A
A d
d a
a n
n e
e t
t 


 

None A
A d
d d
d a
a m
m s
s 


 

None A
A d
d d
d e
e r
r l
l e
e y
y 


 

None A
A d
d d
d i
i n
n a
a l
l l
l 


 

None A
A d
d d
d i
i s
s 


 

None A
A d
d d
d i
i s
s o
o n
n 


 

None A
A d
d d
d l
l e
e y
y 


 

None A
A d
d e
e r
r s
s o
o n
n 


 

None A
A d
d e
e y
y 


 

None A
A d
d k
k i
i n
n s
s 


 

None A
A d
d l
l a
a m
m 


 

None A
A d
d l
l e
e r
r 


 

None A
A d
d r
r o
o l
l 


 

None A
A d
d s
s e
e t
t t
t 


 

None A
A 

B a
a t
t t
t y
y 


 

None B
B a
a x
x t
t e
e r
r 


 

None B
B a
a y
y l
l i
i s
s s
s 


 

None B
B e
e a
a d
d l
l e
e 


 

None B
B e
e a
a l
l 


 

None B
B e
e a
a l
l e
e 


 

None B
B e
e a
a m
m i
i s
s h
h 


 

None B
B e
e a
a n
n 


 

None B
B e
e a
a r
r 


 

None B
B e
e a
a t
t t
t i
i e
e 


 

None B
B e
e a
a t
t t
t y
y 


 

None B
B e
e a
a u
u m
m o
o n
n t
t 


 

None B
B e
e c
c k
k 


 

None B
B e
e d
d f
f o
o r
r d
d 


 

None B
B e
e e
e c
c h
h 


 

None B
B e
e e
e r
r 


 

None B
B e
e g
g u
u m
m 


 

None B
B e
e l
l l
l 


 

None B
B e
e l
l l
l a
a m
m y
y 


 

None B
B e
e n
n f
f i
i e
e l
l d
d 


 

None B
B e
e n
n j
j a
a m
m i
i n
n 


 

None B
B e
e n
n n
n e
e t
t t
t 


 

None B
B e
e n
n s
s o
o n
n 


 

None B
B e
e n
n t
t l
l e
e y
y 


 

None B
B e
e r
r g
g e
e r
r 


 

None B
B e
e r
r n
n a
a r
r d
d 


 

None B
B e
e r
r r
r y
y 


 

None B
B e
e s
s t
t 


 

None B
B e
e t
t h
h e
e l
l l
l 


 

None B
B

 

None C
C o
o n
n n
n e
e e
e l
l y
y 


 

None C
C o
o n
n n
n e
e l
l l
l 


 

None C
C o
o n
n n
n e
e l
l l
l y
y 


 

None C
C o
o n
n n
n o
o l
l l
l y
y 


 

None C
C o
o n
n n
n o
o r
r 


 

None C
C o
o n
n r
r a
a d
d 


 

None C
C o
o n
n r
r o
o y
y 


 

None C
C o
o n
n w
w a
a y
y 


 

None C
C o
o o
o k
k 


 

None C
C o
o o
o k
k e
e 


 

None C
C o
o o
o k
k s
s o
o n
n 


 

None C
C o
o o
o m
m b
b e
e r
r 


 

None C
C o
o o
o m
m b
b e
e s
s 


 

None C
C o
o o
o p
p e
e r
r 


 

None C
C o
o p
p e
e 


 

None C
C o
o p
p e
e l
l a
a n
n d
d 


 

None C
C o
o p
p l
l a
a n
n d
d 


 

None C
C o
o p
p l
l e
e y
y 


 

None C
C o
o r
r b
b e
e t
t t
t 


 

None C
C o
o r
r c
c o
o r
r a
a n
n 


 

None C
C o
o r
r e
e 


 

None C
C o
o r
r l
l e
e t
t t
t 


 

None C
C o
o r
r m
m a
a c
c k
k 


 

None C
C o
o r
r n
n e
e r
r 


 

None C
C o
o r
r n
n i
i s
s h
h 


 

None C
C o
o r
r n
n o
o c
c k
k 


 

None C
C o
o r
r r
r 


 

None C
C

None E
E d
d g
g e
e 


 

None E
E d
d g
g e
e l
l l
l 


 

None E
E d
d g
g e
e r
r t
t o
o n
n 


 

None E
E d
d g
g l
l e
e y
y 


 

None E
E d
d g
g s
s o
o n
n 


 

None E
E d
d k
k i
i n
n s
s 


 

None E
E d
d l
l e
e r
r 


 

None E
E d
d l
l e
e y
y 


 

None E
E d
d l
l i
i n
n g
g t
t o
o n
n 


 

None E
E d
d m
m o
o n
n d
d 


 

None E
E d
d m
m o
o n
n d
d s
s 


 

None E
E d
d m
m o
o n
n d
d s
s o
o n
n 


 

None E
E d
d m
m u
u n
n d
d s
s 


 

None E
E d
d m
m u
u n
n d
d s
s o
o n
n 


 

None E
E d
d n
n e
e y
y 


 

None E
E d
d o
o n
n 


 

None E
E d
d w
w a
a r
r d
d s
s 


 

None E
E d
d w
w i
i c
c k
k 


 

None E
E e
e d
d i
i e
e 


 

None E
E g
g a
a n
n 


 

None E
E g
g e
e r
r t
t o
o n
n 


 

None E
E g
g g
g b
b y
y 


 

None E
E g
g g
g i
i s
s o
o n
n 


 

None E
E g
g g
g l
l e
e s
s t
t o
o n
n 


 

None E
E g
g l
l a
a n
n 


 

None E
E g
g l
l e
e t
t o
o n
n 


 

None E
E g
g l
l i
i n
n 


 

None E
E i
i l
l e
e r
r s


t 


 

None F
F i
i t
t t
t o
o n
n 


 

None F
F i
i t
t z
z g
g e
e r
r a
a l
l d
d 


 

None F
F i
i t
t z
z p
p a
a t
t r
r i
i c
c k
k 


 

None F
F i
i t
t z
z s
s i
i m
m m
m o
o n
n s
s 


 

None F
F l
l a
a c
c k
k 


 

None F
F l
l a
a h
h e
e r
r t
t y
y 


 

None F
F l
l a
a n
n a
a g
g a
a n
n 


 

None F
F l
l a
a n
n d
d e
e r
r s
s 


 

None F
F l
l a
a n
n n
n e
e r
r y
y 


 

None F
F l
l a
a v
v e
e l
l l
l 


 

None F
F l
l a
a x
x m
m a
a n
n 


 

None F
F l
l e
e e
e t
t w
w o
o o
o d
d 


 

None F
F l
l e
e m
m i
i n
n g
g 


 

None F
F l
l e
e t
t c
c h
h e
e r
r 


 

None F
F l
l e
e t
t t
t 


 

None F
F l
l o
o r
r e
e y
y 


 

None F
F l
l o
o s
s s
s 


 

None F
F l
l o
o w
w e
e r
r 


 

None F
F l
l o
o w
w e
e r
r s
s 


 

None F
F l
l o
o y
y d
d 


 

None F
F l
l y
y n
n n
n 


 

None F
F o
o d
d e
e n
n 


 

None F
F o
o g
g g
g 


 

None F
F o
o l
l e
e y
y 


 

None F
F o
o n
n t
t a
a i
i n
n e
e 


 

None F
F o
o r
r a
a 

...

t r
r i
i c
c k
k 


 

None P
P a
a t
t t
t e
e n
n 


 

None P
P a
a t
t t
t e
e r
r s
s o
o n
n 


 

None P
P a
a t
t t
t i
i n
n s
s o
o n
n 


 

None P
P a
a t
t t
t i
i s
s o
o n
n 


 

None P
P a
a t
t t
t o
o n
n 


 

None P
P a
a u
u l
l 


 

None P
P a
a v
v o
o t
t 


 

None P
P a
a w
w s
s o
o n
n 


 

None P
P a
a y
y n
n e
e 


 

None P
P e
e a
a c
c e
e 


 

None P
P e
e a
a c
c h
h 


 

None P
P e
e a
a c
c o
o c
c k
k 


 

None P
P e
e a
a k
k e
e 


 

None P
P e
e a
a l
l 


 

None P
P e
e a
a p
p e
e r
r 


 

None P
P e
e a
a r
r c
c e
e 


 

None P
P e
e a
a r
r s
s 


 

None P
P e
e a
a r
r s
s o
o n
n 


 

None P
P e
e a
a t
t 


 

None P
P e
e c
c k
k 


 

None P
P e
e d
d l
l e
e y
y 


 

None P
P e
e e
e b
b l
l e
e s
s 


 

None P
P e
e e
e l
l 


 

None P
P e
e e
e r
r s
s 


 

None P
P e
e g
g g
g 


 

None P
P e
e i
i g
g n
n e
e 


 

None P
P e
e l
l l
l 


 

None P
P e
e l
l l
l i
i n
n g
g 


 

None P
P e
e m
m b
b e
e r
r t
t

None R
R i
i c
c h
h 


 

None R
R i
i c
c h
h a
a r
r d
d s
s 


 

None R
R i
i c
c h
h a
a r
r d
d s
s o
o n
n 


 

None R
R i
i c
c h
h e
e s
s 


 

None R
R i
i c
c h
h m
m a
a n
n 


 

None R
R i
i c
c h
h m
m o
o n
n d
d 


 

None R
R i
i c
c h
h t
t e
e r
r 


 

None R
R i
i c
c k
k 


 

None R
R i
i c
c k
k a
a r
r d
d 


 

None R
R i
i c
c k
k a
a r
r d
d s
s 


 

None R
R i
i c
c k
k e
e t
t t
t 


 

None R
R i
i c
c k
k e
e t
t t
t s
s 


 

None R
R i
i d
d d
d e
e l
l l
l 


 

None R
R i
i d
d d
d l
l e
e 


 

None R
R i
i d
d d
d l
l e
e r
r 


 

None R
R i
i d
d g
g e
e 


 

None R
R i
i d
d g
g w
w a
a y
y 


 

None R
R i
i d
d g
g w
w e
e l
l l
l 


 

None R
R i
i d
d l
l e
e 


 

None R
R i
i d
d l
l e
e y
y 


 

None R
R i
i g
g b
b y
y 


 

None R
R i
i g
g g
g 


 

None R
R i
i g
g l
l e
e y
y 


 

None R
R i
i l
l e
e y
y 


 

None R
R i
i n
n g
g 


 

None R
R i
i p
p l
l e
e y
y 


 

None R
R i
i p
p p
p i
i n
n 


 

None R
R i
i s
s e


None S
S p
p e
e i
i g
g h
h t
t 


 

None S
S p
p e
e n
n c
c e
e 


 

None S
S p
p e
e n
n c
c e
e r
r 


 

None S
S p
p i
i c
c e
e r
r 


 

None S
S p
p i
i l
l l
l e
e r
r 


 

None S
S p
p i
i n
n k
k s
s 


 

None S
S p
p o
o o
o n
n e
e r
r 


 

None S
S q
q u
u i
i r
r e
e 


 

None S
S q
q u
u i
i r
r e
e s
s 


 

None S
S t
t a
a c
c e
e y
y 


 

None S
S t
t a
a c
c k
k 


 

None S
S t
t a
a f
f f
f 


 

None S
S t
t a
a f
f f
f o
o r
r d
d 


 

None S
S t
t a
a i
i n
n t
t o
o n
n 


 

None S
S t
t a
a m
m p
p 


 

None S
S t
t a
a n
n f
f i
i e
e l
l d
d 


 

None S
S t
t a
a n
n f
f o
o r
r d
d 


 

None S
S t
t a
a n
n l
l e
e y
y 


 

None S
S t
t a
a n
n n
n a
a r
r d
d 


 

None S
S t
t a
a n
n t
t o
o n
n 


 

None S
S t
t a
a r
r k
k 


 

None S
S t
t e
e a
a d
d m
m a
a n
n 


 

None S
S t
t e
e d
d m
m a
a n
n 


 

None S
S t
t e
e e
e l
l 


 

None S
S t
t e
e e
e l
l e
e 


 

None S
S t
t e
e e
e r
r 


 

None S
S t
t e
e e
e r
r e
e 


 

None V
V e
e e
e v
v e
e r
r s
s 


 

None V
V e
e i
i t
t c
c h
h 


 

None V
V e
e l
l d
d s
s 


 

None V
V e
e n
n a
a b
b l
l e
e s
s 


 

None V
V e
e n
n t
t u
u r
r a
a 


 

None V
V e
e r
r d
d o
o n
n 


 

None V
V e
e r
r e
e l
l l
l 


 

None V
V e
e r
r n
n e
e y
y 


 

None V
V e
e r
r n
n o
o n
n 


 

None V
V i
i c
c a
a r
r y
y 


 

None V
V i
i c
c e
e n
n s
s 


 

None V
V i
i c
c k
k a
a r
r s
s 


 

None V
V i
i c
c k
k e
e r
r m
m a
a n
n 


 

None V
V i
i c
c k
k e
e r
r s
s 


 

None V
V i
i c
c k
k e
e r
r y
y 


 

None V
V i
i c
c t
t o
o r
r 


 

None V
V i
i k
k e
e r
r s
s 


 

None V
V i
i l
l l
l i
i g
g e
e r
r 


 

None V
V i
i l
l l
l i
i s
s 


 

None V
V i
i n
n c
c e
e 


 

None V
V i
i n
n c
c e
e n
n t
t 


 

None V
V i
i n
n e
e 


 

None V
V i
i n
n e
e r
r 


 

None V
V i
i n
n e
e s
s 


 

None V
V i
i n
n e
e y
y 


 

None V
V i
i n
n i
i c
c o
o m
m b
b e
e 


 

None V
V i
i n
n n
n y
y 


 

None V
V i
i n
n t
t

e y
y 


 

None Y
Y e
e a
a t
t e
e s
s 


 

None Y
Y e
e a
a t
t m
m a
a n
n 


 

None Y
Y e
e l
l d
d o
o n
n 


 

None Y
Y e
e o
o m
m a
a n
n 


 

None Y
Y e
e o
o m
m a
a n
n s
s 


 

None Y
Y e
e t
t m
m a
a n
n 


 

None Y
Y e
e u
u n
n g
g 


 

None Y
Y o
o m
m a
a n
n 


 

None Y
Y o
o m
m k
k i
i n
n s
s 


 

None Y
Y o
o r
r k
k 


 

None Y
Y o
o r
r k
k e
e 


 

None Y
Y o
o r
r s
s t
t o
o n
n 


 

None Y
Y o
o u
u l
l d
d e
e n
n 


 

None Y
Y o
o u
u n
n g
g 


 

None Y
Y o
o u
u n
n g
g e
e 


 

None Y
Y o
o u
u n
n i
i s
s 


 

None Y
Y o
o u
u s
s s
s o
o u
u f
f 


 

None Y
Y u
u l
l e
e 


 

None Y
Y u
u s
s u
u f
f 


 

None Z
Z a
a o
o u
u i
i 


 



In [ ]:
# ok, so now let's just do the stuff
# the RNN actually makes one-character over predictions! 

